## Imports

In [ ]:
!pip install cma

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import the locally stored `pysocialforce` model
import sys
sys.path.append('/content/drive/MyDrive/AC297R/Code')
import pysocialforce as psf

In [3]:
from pathlib import Path
import numpy as np
from itertools import islice
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.patches as patches
import pickle
import cma
import cv2

## Load data and set global parameters

In [4]:
nyc_data = np.load("/content/drive/MyDrive/AC297R/Code/state_history.npy")
n_pedestrians = np.load("/content/drive/MyDrive/AC297R/Code/pred_counts.npy")

In [5]:
# These are the baseline paramters for the Social Force Model
base_config_file = {
                  "scene": {
                      "enable_group": True,
                      "agent_radius": 0.35,
                      "max_speed_multiplier": 1.3
                  },
                  "desired_force": {
                      "factor": 1.0,
                      "goal_threshold": 0.2,
                      "relaxation_time": 0.5
                  },
                  "social_force": {
                      "factor": 5.1,
                      "lambda_importance": 2.0,
                      "gamma": 0.35,
                      "n": 2,
                      "n_prime": 3
                  },
                  "obstacle_force": {
                      "factor": 10.0,
                      "sigma": 0.4,
                      "threshold": 3.0
                  },
                  "group_coherence_force": {
                      "factor": 3.0
                  },
                  "group_repulsive_force": {
                      "factor": 1.0,
                      "threshold": 0.55
                  },
                  "group_gaze_force": {
                      "factor": 4.0,
                      "fov_phi": 90.0
                  }
              }

n_timesteps = 375
n_peds = n_pedestrians[0]

In [9]:
real_counts = nyc_data / nyc_data.sum(axis=(1, 2), keepdims=True)

for t in range(real_counts.shape[0]):
    real_counts[t] = real_counts[t] * n_pedestrians[t]

real_counts.shape

(375, 10, 8)

In [ ]:
# Consider the (smoothed) number of people in the top 2 pixels of the image. We assume that this is the number of new pedestrians entering in a given frame
top_counts = []

def moving_average(data, window):
    weights = np.ones(window) / window
    return np.convolve(data, weights, mode='valid')

for t in range(real_counts.shape[0]):
    top_counts.append(real_counts[t, -1, :].sum())

top_counts = np.array(top_counts)
smooth_counts = moving_average(top_counts, 50) // 7.5
plt.plot(smooth_counts)
plt.show()

In [11]:
initial_state = []
new_pedestrians = {}

for x in range(8):
  for y in range(10):
    for ped in range(int((real_counts[0].T)[x][y] + 0.5)):
      px = np.random.uniform(x*14, (x+1)*14)
      while not ((px >= 0 and px <= 45) or (px >= 60 and px <= 112)):
        px = np.random.uniform(x*14, (x+1)*14)

      py = 150 - np.random.uniform(y*15, (y+1)*15)
      while not ((py >= 0 and py <= 50) or (py >= 60 and py <= 150)):
        py = 150 - np.random.uniform(y*15, (y+1)*15)

      gx = np.random.normal(px, 10)
      while not ((gx >= 0 and gx <= 55) or (gx >= 83 and gx <= 112)):
        gx = np.random.normal(px, 10)

      gy = 0

      vx = gx - px
      vy = gy - py

      k = np.sqrt((vx ** 2 + vy ** 2) / (1.78 ** 2)) # Rescale to average jogging speed
      vx = vx / k
      vy = vy / k

      state_list = [
          px, py, vx, vy, gx, gy
      ]

      # if t == 0:
      initial_state.append(state_list)
      # else:
      #     if entry_time not in new_pedestrians:
      #         new_pedestrians[entry_time] = []
      #     new_pedestrians[entry_time].append(state_list)

for t in range(1, len(smooth_counts)):
  new_pedestrians[t] = []
  for _ in range(int(smooth_counts[t])):
    px = np.random.uniform(0, 112)
    py = 150
    gx = np.random.normal(px, 10)
    while not ((gx >= 0 and gx <= 55) or (gx >= 83 and gx <= 112)):
        gx = np.random.normal(px, 10)
    gy = 0
    vx = gx - px
    vy = gy - py

    k = np.sqrt((vx ** 2 + vy ** 2) / (1.78 ** 2)) # Rescale to average jogging speed
    vx = vx / k
    vy = vy / k

    state_list = [
        px, py, vx, vy, gx, gy
    ]

    new_pedestrians[t].append(state_list)


initial_state = np.array(initial_state)
n_peds = initial_state.shape[0]

groups = None

x_min = 45
x_max = 60
y_min = 50
y_max = 60

bao_stand = [[x_min, x_max, y_min, y_min], [x_min, x_max, y_max, y_max], [x_min, x_min, y_min, y_max], [x_max, x_max, y_min, y_max]]

x_min = 55
x_max = 83
y_min = 0
y_max = 5

building = [[x_min, x_max, y_min, y_min], [x_min, x_max, y_max, y_max], [x_min, x_min, y_min, y_max], [x_max, x_max, y_min, y_max]]

obs = bao_stand + building

# Utils

In [6]:
def traj_to_density_map(data, timesteps=n_timesteps):
    """
    Converts a dataset of pedestrian trajectories into a density map.

    The map represents the number of pedestrians present in various grid tiles at different timesteps. 
    The function creates this grid defined by the number of rows and columns, and counts the number of 
    pedestrians in each grid tile at each timestep based on their x, y coordinates.

    Parameters:
    - data (np.ndarray): A three-dimensional NumPy array where each element data[t, p, :2] contains 
      the x and y coordinates of the p-th pedestrian at the t-th timestep.
    - timesteps (int): The number of timesteps to process.

    Returns:
    - np.ndarray: A three-dimensional integer array of shape (timesteps, grid_cols, grid_rows) 
      where each element at a given timestep represents the count of pedestrians in a grid tile.
    
    Notes:
    - Ensure that the data provided has the dimensions expected (timesteps, n_peds, 2), where `n_peds` 
      is the number of pedestrians.
    - The function assumes a fixed grid size with predefined tile dimensions and an area bounds (0 to 150 units).
    """

    # Define constants
    grid_rows = 8
    grid_cols = 10
    tile_height = 14
    tile_width = 15

    # Initialize grid to store pedestrian counts
    grid = np.zeros((timesteps, grid_cols, grid_rows), dtype=int)

    # Iterate over timesteps and pedestrians to count pedestrians in each grid tile
    for t in range(timesteps):
        for p in range(n_peds):
            x, y = data[t, p, :2]
            if x > 0 and y > 0:
                grid_y = int(min((150 - y) // tile_width, grid_cols - 1))
                grid_x = int(min(x // tile_height, grid_rows - 1))
                grid[t, grid_y, grid_x] += 1

    return grid

In [12]:
def kl_divergence(state1, state2):
    """
    Calculates the Kullback-Leibler (KL) divergence between two distributions.

    This function computes the KL divergence from state2 to state1, adjusting any zero values in both 
    distributions to a small number (1e-5) to prevent division by zero errors.

    Parameters:
    - state1 (np.ndarray): The first density distribution (reference distribution).
    - state2 (np.ndarray): The second density distribution (distribution to be compared).

    Returns:
    - float: The KL divergence value indicating how different state2 is from state1.

    Notes:
    - It is assumed both input arrays are non-negative and represent density distributions.
    """
    # Ensure non-zero bins to avoid division by zero in KL divergence computation
    hist1 = np.where(state1 == 0, 1e-5, state1)
    hist2 = np.where(state2 == 0, 1e-5, state2)

    # plt.figure()
    # plt.matshow(hist1, cmap='Reds')
    # plt.colorbar()
    # plt.show()

    # plt.figure()
    # plt.matshow(hist2, cmap='Reds')
    # plt.colorbar()
    # plt.show()

    # Compute KL Divergence
    mask = (hist1 > 0) & (hist2 > 0)
    kl_div = entropy(hist1[mask], hist2[mask])

    return kl_div

In [7]:
def compute_total_loss(real_data, sim_data):
  """
  Computes the total KL divergence loss between real and simulated data across multiple timesteps.

  This function calculates the total KL divergence loss by comparing real data with simulated data at each timestep.
  It adjusts the total loss by adding a penalty if the KL divergence is zero, promoting better simulation fidelity.

  Parameters:
  - real_data (np.ndarray): A three-dimensional array of shape (timesteps, grid_cols, grid_rows) representing real data over multiple timesteps.
  - sim_data (np.ndarray): A three-dimensional array of trajectories representing simulated data to be compared with real data.

  Returns:
  - float: The total KL divergence loss accumulated over all timesteps.
  """
  total_loss = 0

  simulated_coords = traj_to_density_map(sim_data)

  for t in range(real_data.shape[0]):
      real_coords_t = real_data[t]
      simulated_coords_t = simulated_coords[t]
      assert simulated_coords_t.shape == real_coords_t.shape

      # Compute KL divergence for this timestep
      kl_div_t = kl_divergence(real_coords_t, simulated_coords_t)
      total_loss = total_loss + kl_div_t if kl_div_t > 0 else total_loss + 10

  print(f"Total KL Divergence Loss over all timesteps: {total_loss}")
  return total_loss

In [8]:
def restructure_config(flat_params):
    """
    Update the base configuration dictionary with values from flat_params. Only keys existing in flat_params
    will be updated in base_config.

    Parameters:
    - base_config: The original configuration dictionary to be updated.
    - flat_params: Dictionary with flat parameter names and their new values, e.g., 'desired_force__factor': 0.75.

    Returns:
    - Updated base configuration with new values from flat_params.
    """
    base_config = base_config_file.copy()
    for flat_key, new_value in flat_params.items():
        # Split the flat key by '__' to get the hierarchy levels
        keys = flat_key.split('__')
        current_level = base_config

        # Iterate over the keys to reach the correct level in the base_config
        for key in keys[:-1]:
            if key in current_level:
                current_level = current_level[key]
            else:
                # If the key is not found at the current level, skip the rest of the update for this flat_key
                break
        else:  # This else belongs to the for loop, it executes if the loop completes normally (no break)
            # Update the value if the final key is in the current level of base_config
            if keys[-1] in current_level:
                current_level[keys[-1]] = float(new_value)

    return base_config

In [ ]:
def generate_pbounds(config, prefix=''):
    """
    Generates parameter bounds from a configuration dictionary.

    This function constructs a dictionary of parameter bounds based on the provided configuration.
    For numeric values (int or float), it sets the bounds to +/-25% of the given value. Nested 
    dictionaries are processed recursively, and parameter names are constructed to reflect their hierarchy.

    Parameters:
    - config (dict): A dictionary containing configuration parameters, which may include nested dictionaries.
    - prefix (str, optional): A prefix for parameter names, used to denote hierarchy in nested dictionaries.

    Returns:
    - dict: A dictionary where keys are parameter names and values are tuples representing the bounds (min, max).
    """
    pbounds = {}
    for key, value in config.items():
        # Construct the parameter name with hierarchy, e.g., "scene__agent_radius"
        param_name = f"{prefix}__{key}" if prefix else key

        if isinstance(value, dict):
            # Recursively handle nested dictionaries
            pbounds.update(generate_pbounds(value, param_name))
        elif isinstance(value, (int, float)):
            # Calculate bounds with +/-25% range for numeric values
            pbounds[param_name] = (value * 0.75, value * 1.25)

    return pbounds

In [13]:
def run_simulation(parameters, initial_state=initial_state, obs = obs, groups = groups):
  """
  Runs a pedestrian simulation given initial conditions, groups, and obstacles, using specified parameters.

  This function initializes a simulation with the provided initial state of pedestrians and environmental 
  constraints like groups and obstacles. It progresses the simulation for a set number of timesteps, 
  and then returns the history of states for the simulation up to those timesteps.

  Parameters:
  - parameters (dict): Dictionary containing simulation parameters.
  - initial_state (np.ndarray): Initial states of the pedestrians in the simulation.
  - obs (list): A list of obstacles to be considered in the simulation environment.
  - groups (list): A list representing groups of pedestrians to model social or group dynamics.

  Returns:
  - np.ndarray: A three-dimensional array representing the history of pedestrian states across the specified timesteps.
  """
  s = psf.Simulator(
    initial_state,
    groups=groups,
    obstacles=obs,
    config_file=parameters,
  )
  s.step(n_timesteps, new_pedestrians=new_pedestrians)
  simul_history = s.get_states()[0][:n_timesteps, :, :2]
  return simul_history

## Optimization Routine (using CMA-ES)

In [ ]:
params = {
    "scene": {
        "max_speed_multiplier": 1.3
    },
    "desired_force": {
        "factor": 1.0,
    },
    "social_force": {
        "factor": 5.1,
    },
    "obstacle_force": {
        "factor": 10.0,
    }
}

# Initial guess for CMA-ES, set to the midpoint of the bounds
pbounds = generate_pbounds(params)
x0 = [(low + high) / 2 for _, (low, high) in pbounds.items()]

In [14]:
# Global variable that stores the simulation history (used for debugging)
s = None

def objective_function(x):
    """
    Objective function for optimization, which evaluates the loss for a given set of simulation parameters.

    This function is designed to be used with an optimizer that minimizes the loss. It takes a list of parameter values,
    constructs a configuration dictionary, runs a pedestrian simulation with this configuration, and computes the loss
    by comparing the simulation output to real data. The loss is computed as the total KL divergence between the
    simulated data and the real counts of pedestrians.

    Parameters:
    - x (list): A list of parameter values corresponding to the bounds in `pbounds`.

    Returns:
    - float: The computed loss for the given parameters.

    Notes:
    - `pbounds` should be defined globally or in the outer scope where `objective_function` is called.
    - This function modifies a global variable `s` to store the simulation data for external use.
    """
    # takes in parameters, outputs loss
    flat_params = dict(zip(pbounds.keys(), x))
    structured_config = restructure_config(flat_params)
    print(flat_params)
    sim_data = run_simulation(structured_config) # n_timesteps, n_peds, 2
    global s
    s = sim_data
    loss = compute_total_loss(real_counts, sim_data)
    return loss

In [16]:
# Define a callback function to store optimization data
class OptimizationLogger(object):
    def __init__(self):
        self.best_scores = []
        self.param_history = []

    def __call__(self, es):
        # Store the best score and the current best solution
        self.best_scores.append(es.result.fbest)
        self.param_history.append(es.result.xbest)

# Initialize the logger
logger = OptimizationLogger()

In [17]:
# A single scalar value for sigma0. Adjust this based on your parameter scales and domain knowledge
sigma0 = 0.1  # Or some other scalar value appropriate for your problem
options = {
    'bounds': list(zip(*pbounds.values())),
    'maxiter': 10,  # Maximum number of iterations
    'tolfun': 1e-6,  # Tolerance in function value change for convergence
    'tolx': 1e-6,    # Tolerance in parameter change for convergence
    'verb_disp': 1,  # Frequency of displayed information (1 for every iteration, for example)
    'verb_log': 1,
}
es = cma.CMAEvolutionStrategy(x0, sigma0, options)
# Run CMA-ES optimization
es.optimize(objective_function, callback=[logger])

# Get the best solution
best_solution = es.result.xbest
best_score = es.result.fbest

# Convert the best solution back to the structured configuration
best_params_flat = dict(zip(pbounds.keys(), best_solution))
best_params_structured = restructure_config(best_params_flat)

print(f"Best parameters: {best_params_structured}")
print(f"Best score: {best_score}")

/usr/local/lib/python3.10/dist-packages/cma/evolution_strategy.py:1901: UserWarning: Sampling standard deviation i=4 at iteration 0 change by 0.666640000533326 to stds[4]=0.06666666666666664
  warnings.warn("Sampling standard deviation i={} at iteration {}"


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=159448, Thu Apr 25 00:42:38 2024)
{'scene__max_speed_multiplier': 1.436596473733872, 'desired_force__factor': 0.8329039386952142, 'social_force__factor': 4.971183249406289, 'obstacle_force__factor': 9.94408507374884, 'obstacle_force__sigma': 0.42830621569076277}
Total KL Divergence Loss over all timesteps: 3095.8144613101
{'scene__max_speed_multiplier': 1.3773661071044483, 'desired_force__factor': 1.1033454878198294, 'social_force__factor': 5.088109759110398, 'obstacle_force__factor': 10.034751160924053, 'obstacle_force__sigma': 0.42134398095444453}
Total KL Divergence Loss over all timesteps: 562.4384651100488
{'scene__max_speed_multiplier': 1.2251514875454206, 'desired_force__factor': 1.0507673312008206, 'social_force__factor': 5.004985374519435, 'obstacle_force__factor': 9.84031759402376, 'obstacle_force__sigma': 0.3729445980071711}
Total KL Divergence Loss over all timesteps: 3059.15596511595
{'scene__max_speed_multiplier': 1.

In [18]:
# Save the optimal parameters to a pickle file
with open('/content/drive/MyDrive/AC297R/Code/best_params_v3.0.pickle', 'wb') as handle:
    pickle.dump(best_params_structured, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Visualize simulation with optimized parameters

In [ ]:
best_params = best_params_structured

# # Or, load saved parameters from a past optimization routine (change the file path as needed)
# with open('/content/drive/MyDrive/AC297R/Code/best_params_v3.0.pickle', 'rb') as file:
#     best_params = pickle.load(file)

In [ ]:
# Run the simulation with the optimized parameters
best_simulation = run_simulation(best_params)

In [39]:
# Plot/animate trajectories using pysocialforce plotting functions

with psf.plot.SceneVisualizer(s, "/content/drive/MyDrive/AC297R/Code") as sv:
    # sv.animate(density=True)
    sv.plot()

In [ ]:
# Plot the trajectories from the simulation with the optimized parameters

fig, ax = plt.subplots()

# Loop over each pedestrian
for i in range(best_simulation.shape[1]):
    # Extract the x and y coordinates for the ith pedestrian
    x_coords = np.array(best_simulation[:, i, 0])
    y_coords = np.array(best_simulation[:, i, 1])

    # Filter out the points where the pedestrian has not entered or has exited (x, y) == (0, 0)
    valid_points = (x_coords != 0) | (y_coords != 0)

    # Get the indices of the valid points
    valid_indices = np.where(valid_points)[0]

    if valid_indices.size > 0:
        # Plot the trajectory for the pedestrian and capture the line color
        line, = plt.plot(x_coords[valid_points], y_coords[valid_points])
        line_color = line.get_color()

        # Plot the start point (triangle) using the line's color
        start_idx = valid_indices[0]
        plt.scatter(x_coords[start_idx], y_coords[start_idx], marker='^', s=50, color=line_color)

        # Plot the end point (square) using the line's color
        end_idx = valid_indices[-1]
        plt.scatter(x_coords[end_idx], y_coords[end_idx], marker='s', s=50, color=line_color)


bao_obs = patches.Rectangle((45, 50), 15, 10, linewidth=2, edgecolor='k', facecolor='k')
bao_obs.set_zorder(10)
ax.add_patch(bao_obs)

building_obs = patches.Rectangle((55, 0), 28, 4, linewidth=2, edgecolor='k', facecolor='k')
building_obs.set_zorder(10)
ax.add_patch(building_obs)

# Set plot labels and title
plt.xlim(0,112)
plt.ylim(0,150)
plt.title('Pedestrian Trajectories')
plt.grid(True)
plt.show()

In [31]:
# Create the animated real and predicted density maps

def matrix_to_xy_pairs(matrix, sampling_factor=100):
    """
    Converts a density map into a list of (x, y) pairs based on the matrix values.

    Each non-zero element in the matrix is represented by a number of (x, y) pairs proportional 
    to its value relative to the maximum matrix value, scaled by a sampling factor.

    Parameters:
    - matrix (np.ndarray): A 2D array representing a matrix whose elements indicate density or count.
    - sampling_factor (int): The factor used to determine the number of samples per element; default is 100.

    Returns:
    - np.ndarray: A 2D array where each row is an (x, y) pair corresponding to a matrix element.

    Notes:
    - The function returns an empty array if all matrix values are zero.
    """
    max_value = np.max(matrix)
    rows, cols = matrix.shape
    xy_pairs = []

    if max_value == 0:  # Add a condition to handle the case where there are no non-zero values
        return np.empty((0, 2), dtype=int)  # Return an empty 2D array

    for i in range(rows):
        for j in range(cols):
            try:
                num_samples = int((matrix[i, j] / max_value) * sampling_factor)
            except:
                num_samples = 0
            xy_pairs.extend([(i, j)] * num_samples)

    return np.array(xy_pairs, dtype=int).reshape(-1, 2)


def animate_density_map(real_plot_data, simul_plot_data):
    """
    Animates two sets of density map data for comparison between real and simulated observations.

    This function creates a Matplotlib animation with two subplots, displaying the density maps of 
    real and simulated data over time. It uses Kernel Density Estimation (KDE) to generate the plots.

    Parameters:
    - real_plot_data (List[np.ndarray]): A list of 2D numpy arrays containing the real data.
    - simul_plot_data (List[np.ndarray]): A list of 2D numpy arrays containing the simulated data.

    Returns:
    - matplotlib.animation.FuncAnimation: The animation object displaying the real and simulated density maps.
    """
    assert len(real_plot_data) == len(simul_plot_data)

    # Prepare the figure for animation with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))
    plt.close(fig)  # Avoid displaying the static plot in some environments
    title = fig.suptitle('Frame 0', fontsize=12)

    # Initialization function for the animation
    def init():
        ax1.clear()
        ax2.clear()
        sns.kdeplot(x=[0], y=[0], ax=ax1)  # Dummy plot for initialization on ax1
        sns.kdeplot(x=[0], y=[0], ax=ax2)  # Dummy plot for initialization on ax2
        ax1.set_title('Real')  # Title for ax1
        ax2.set_title('Simulated')  # Title for ax2

        # Create and add patches separately for each axes
        bao_obs1 = patches.Rectangle((45/14, 9-50/15), 15/14, -10/15, linewidth=2, edgecolor='k', facecolor='k')
        bao_obs2 = patches.Rectangle((45/14, 9-50/15), 15/14, -10/15, linewidth=2, edgecolor='k', facecolor='k')
        building_obs1 = patches.Rectangle((55/14, 9), 28/14, -4/15, linewidth=2, edgecolor='k', facecolor='k')
        building_obs2 = patches.Rectangle((55/14, 9), 28/14, -4/15, linewidth=2, edgecolor='k', facecolor='k')

        bao_obs1.set_zorder(10)
        bao_obs2.set_zorder(10)
        building_obs1.set_zorder(10)
        building_obs2.set_zorder(10)

        ax1.add_patch(bao_obs1)
        ax1.add_patch(building_obs1)
        ax2.add_patch(bao_obs2)
        ax2.add_patch(building_obs2)

    # Update function for the animation
    def update(frame):
        print(frame)
        ax1.clear()  # Clear previous plot on ax1
        ax2.clear()  # Clear previous plot on ax2

        real_xy_pairs = matrix_to_xy_pairs(real_plot_data[frame], sampling_factor=500)  # Convert matrix to xy pairs
        simul_xy_pairs = matrix_to_xy_pairs(simul_plot_data[frame], sampling_factor=500)  # Convert matrix to xy pairs

        sns.kdeplot(x=real_xy_pairs[:, 1], y=real_xy_pairs[:, 0], ax=ax1, fill=True, cmap="Reds")
        sns.kdeplot(x=simul_xy_pairs[:, 1], y=simul_xy_pairs[:, 0], ax=ax2, fill=True, cmap="Reds")
        title.set_text(f'Frame {frame}')  # Update the suptitle text
        ax1.set_title('Real')  # Title for ax1
        ax2.set_title('Simulated')  # Title for ax2

        # Create and add patches separately for each axes
        bao_obs1 = patches.Rectangle((45/14, 9-50/15), 15/14, -10/15, linewidth=2, edgecolor='k', facecolor='k')
        bao_obs2 = patches.Rectangle((45/14, 9-50/15), 15/14, -10/15, linewidth=2, edgecolor='k', facecolor='k')
        building_obs1 = patches.Rectangle((55/14, 9), 28/14, -4/15, linewidth=2, edgecolor='k', facecolor='k')
        building_obs2 = patches.Rectangle((55/14, 9), 28/14, -4/15, linewidth=2, edgecolor='k', facecolor='k')

        bao_obs1.set_zorder(10)
        bao_obs2.set_zorder(10)
        building_obs1.set_zorder(10)
        building_obs2.set_zorder(10)

        ax1.add_patch(bao_obs1)
        ax1.add_patch(building_obs1)
        ax2.add_patch(bao_obs2)
        ax2.add_patch(building_obs2)

        # Setting identical limits and titles for both axes
        for ax in [ax1, ax2]:
            ax.set_xlim(0, 7)
            ax.set_ylim(0, 9)
            ax.invert_yaxis()  # Invert y-axis to match the plot in the video

    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(real_plot_data), init_func=init, blit=False, repeat=True)
    return ani

In [ ]:
simulation_plot_data = traj_to_density_map(best_simulation)
simulation_ani = animate_density_map(real_counts, simulation_plot_data)
HTML(simulation_ani.to_html5_video())

In [ ]:
simulation_ani.save('/content/drive/MyDrive/AC297R/Code/animation_t=0-350_v3.0.mp4', fps=5, writer='ffmpeg')
print("Animation saved as 'animation_t=0-350_v3.0.mp4'.")

In [34]:
def animate_projected_map(real_plot_data, simul_plot_data, video_frames):
    """
    Creates an animation showing side-by-side comparisons of video frames and corresponding real and 
    simulated density maps, with a perspective transformation applied to the density maps.

    This function generates an animation with three vertical subplots: the original video frame, 
    the real data density map, and the simulated data density map. Each density map is transformed 
    using a predefined perspective matrix to match the video frame perspective.

    Parameters:
    - real_plot_data (List[np.ndarray]): A list of 2D numpy arrays containing real density data.
    - simul_plot_data (List[np.ndarray]): A list of 2D numpy arrays containing simulated density data.
    - video_frames (List[np.ndarray]): A list of video frames as numpy arrays.

    Returns:
    - matplotlib.animation.FuncAnimation: The animation object displaying the video alongside the real 
      and simulated transformed density maps.

    Notes:
    - The input lists must all be of the same length.
    - The perspective transform assumes specific source and destination points which may need 
      adjustment depending on the specifics of the scene geometry.
    """
    assert len(real_plot_data) == len(simul_plot_data) == len(video_frames)

    # Prepare the figure for animation with three subplots stacked vertically
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(24, 16))
    fig.subplots_adjust(hspace=0.5)
    plt.close(fig)  # Avoid displaying the static plot in some environments
    title = fig.suptitle('Frame 0')

    # Compute the perspective transform matrix
    src = np.float32([
        [0, 0], [88/14, 0], [112/14, 10], [0, 10]
    ])
    dst = np.float32([
        [800, 300], [1150, 400], [1000, 680], [340, 400]
    ])
    M = cv2.getPerspectiveTransform(src, dst)

    # Apply the perspective transformation to plot data
    real_frames = [cv2.warpPerspective(np.float32(frame), M, (1280, 720)) for frame in real_plot_data]
    simul_frames = [cv2.warpPerspective(np.float32(frame), M, (1280, 720)) for frame in simul_plot_data]

    # Initialization function for the animation
    def init():
        ax1.imshow(video_frames[0])  # Display the first frame of the video on ax1
        ax2.matshow(real_frames[0], cmap="Reds")  # Dummy plot for initialization on ax2
        ax3.matshow(simul_frames[0], cmap="Reds")  # Dummy plot for initialization on ax3
        ax1.set_title('Video')  # Title for ax1
        ax2.set_title('Real Density')  # Title for ax2
        ax3.set_title('Simulated Density')  # Title for ax3

        ax1.xaxis.set_ticks_position('top')
        ax1.xaxis.set_label_position('top')

    # Update function for the animation
    def update(frame):
        ax1.imshow(video_frames[frame])  # Update video frame on ax1
        ax2.matshow(real_frames[frame], cmap="Reds")  # Update real data frame on ax2
        ax3.matshow(simul_frames[frame], cmap="Reds")  # Update simulated data frame on ax3
        title.set_text(f'Frame {frame}')  # Update the suptitle text
        return [ax1, ax2, ax3]

    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(real_plot_data), init_func=init, blit=False, repeat=True)
    return ani

In [ ]:
video_path = '/content/drive/MyDrive/AC297R/earthcam_ny_video.mp4'

times = np.arange(17, 47, 0.08)

video_frames = []
cap = cv2.VideoCapture(video_path)

for desired_time in times:
    # Get video frame rate
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Calculate the frame number based on the desired time and frame rate
    frame_number = int(desired_time * fps)

    # Set the video position to the desired frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()

    # Convert the frame from BGR to RGB (for correct color rendering in matplotlib)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    video_frames.append(frame_rgb)

video_frames = np.array(video_frames)
cap.release()

simulation_plot_data = traj_to_density_map(best_simulation)
ani = animate_projected_map(real_counts, simulation_plot_data, video_frames)
HTML(ani.to_html5_video())

In [ ]:
ani.save('/content/drive/MyDrive/AC297R/Code/animation_t=0-350_withvid_v3.0.mp4', fps=5, writer='ffmpeg')
print("Animation saved as 'animation_t=0-350_withvid_v3.0.mp4'.")

In [ ]:
# View how KL deivergence evolves across timesteps

def plot_KLdiv_vs_time(real_data, sim_data, baseline_data):
    """
    Plots the difference in KL divergence over time between two simulation datasets and real data.

    This function computes the KL divergence from the real data to both the simulated data and 
    the baseline data at each timestep, and then plots the difference in these divergences over time.

    Parameters:
    - real_data (np.ndarray): A 3D numpy array where each row represents real data at a timestep.
    - sim_data (np.ndarray): A 3D numpy array where each row represents simulated data at a timestep.
    - baseline_data (np.ndarray): A 3D numpy array where each row represents baseline simulation data at a timestep.
    """
    num_timesteps = real_data.shape[0]
    kl_div = np.zeros(num_timesteps)
    kl_div_b = np.zeros(num_timesteps)
    for t in range(num_timesteps):
        real_coords_t = real_data[t]
        simulated_coords_t = sim_data[t]
        baseline_coords_t = baseline_data[t]
        # Compute KL divergence for this timestep
        kl_div[t] = kl_divergence(real_coords_t, simulated_coords_t)
        kl_div_b[t] = kl_divergence(real_coords_t, baseline_coords_t)

    # plt.plot(range(1, num_timesteps), kl_div[1:], label='Optimized')
    # plt.plot(range(1, num_timesteps), kl_div_b[1:], label="Baseline")
    plt.plot(range(1, num_timesteps), kl_div[1:] - kl_div_b[1:])
    plt.xlabel('Timestep')
    plt.ylabel('KL Divergence')
    plt.title('KL Divergence vs. Time for the Optimized Parameters')
    plt.legend()
    plt.show()

In [ ]:
# Run the simulation with the baseline configuration
baseline_simulation = run_simulation(base_config_file)

# Plot KL divergence for each timestep after t = 0
simulation_plot_data = traj_to_density_map(best_simulation)
baseline_simulation_plot_data = traj_to_density_map(baseline_simulation)
plot_KLdiv_vs_time(real_counts, simulation_plot_data, baseline_simulation_plot_data)